# Multilayer Perceptron with Keras

The goal of this notebook is to introduce the Perceptron and Multilayer Perceptron as a first hands-on problem to Neural Networks while explaining a well-known library for ANNs: Keras.

With that in mind, we will dig too deep into understanding the distribution of the data in the California Housing dataset.


## Importing Libraries and Dataset

In [ ]:
# sklearn 
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# numpy
import numpy as np 

# pandas
import pandas as pd

# images 
from IPython import display


In [ ]:
housing = fetch_california_housing()


## Dataset Information
The only data analysis that we will cover in this notebook is the report given by Scikit Learn.

In [ ]:
print(housing.DESCR)

# Perceptron

A Perceptron can be seen as one of the simplest Artificial Neural Networks. It is composed by a single layer of artificial neurons called *Threshold Logic Unit* (TLU). This layer computes a weighted sum of its inputs 

$$(z = w_1 x_1 + w_2 x_2 + ⋯ + w_n x_n = x^T w),$$

then applies a step function to that sum and outputs the result: 

$$h_w(x) = step(z),\,where\, z = x^T w.$$

A single TLU can be used for simple linear binary classification. It computes a linear combination of the inputs and if the result exceeds a threshold, it outputs the positive class. Otherwise it outputs the negative class (just like a Logistic Regression classifier or a linear SVM). Training a TLU, in this case, means finding the right values for $w_0$, $w_1$ and $w_2$.


![TLU](../docs/TLU.png)

When all the neurons in a layer are connected to every neuron in the previous layer (i.e., its input neurons), it is called a fully connected layer or a dense layer. To represent the fact that each input is sent to every TLU, it is common to draw special passthrough neurons called input neurons: they just output whatever input they are fed. All the input neurons from the input layer. Moreover, an extra bias parameter is generally added $(x_0 = 1)$: it is typically represented using a special type of neuron called a bias neuron, which just outputs 1 all the time.

![](../docs/perceptron.png)


Scikit-Learn provides a Perceptron class that implements a single TLU network that can be used to classify a single data point since it only returns a binary value.

In [ ]:
# Split dataset 
X = housing.data
y = housing.target

**Let's infer if a house will have an evaluation greater than 2k.** 

We can do this by creating a binary label that will tell the perceptron if the house has indeed a value greater than 2k.

In [ ]:
y_label = (y > 1).astype(np.int)

In [ ]:
from sklearn.linear_model import Perceptron
per = Perceptron(random_state=2) # to use always the same seed
per.fit(X,y_label)
print(per.score(X,y_label))

This prediction can be improved if the hyperparameters were tuned, nonetheless is important to mention that this is a single, that executes fast. To have better results we need something more.

**Note that contrary to Logistic Regression classifiers, Perceptrons do not output a class probability rather, they just make predictions based on a hard threshold.**

## 

# Multilayer Perceptron
**It turns out that some of the limitations of Perceptrons can be eliminated by stacking multiple Perceptrons. The resulting ANN is called a Multi-Layer Perceptron (MLP).**

A MLP is composed of one (passthrough) input layer, one or more layers of TLUs called hidden layers, and one final layer of TLUs called the output layer

![mlp](../docs/mlp.png)

When an ANN contains a deep stack of hidden layers, it is called a Deep Neural Network (DNN). The field of Deep Learning studies DNNs, and more generally models containing deep stacks of computations. However, many people talk about Deep Learning whenever neural networks are involved (even shallow ones).

For this algorithm to work properly, the authors made a key change to the MLP’s architecture: they replaced the step function (showed in the Perceptron above) known as activation function with the logistic function, $\sigma(z) = 1 / (1 + exp(–z))$.

**But how this all works?**
Well, you first break the training data into mini-batches. Then, for each training instance, the backpropagation algorithm first makes a prediction (forward pass), measures the error. Afterwards it goes through each layer in reverse to measure the error contribution from each connection (reverse pass), and finally slightly tweaks the connection weights to reduce the error (Gradient Descent step).

**There are many activation functions three of the most popular are:**

-  *Hyperbolic Tangent Function:* $tanh(Z)=2\sigma(2z)-1$

- *Rectified Linear Unit Function:* $ReLU(Z) = max(0,z)$

- *Sigmoid Function:* $\sigma(z) = 1 / (1 + exp(–z))$ 


![activation functions](../docs/actv-functions.png)

The activation functions are a very important spet in the network because if you chain several linear transformations, all you get is a linear transformation. So if you don’t have some non-linearity between layers, then even a deep stack of layers is equivalent to a single layer: you cannot solve very complex problems with that.

## Regression With MLPs

If you want to predict a single value (e.g., the price of a house given many of its features), then you just need a single output neuron: its output is the predicted value. For multivariate regression (i.e., to predict multiple values at once), you need one output neuron per output dimension.

In general, when building an MLP for regression, you do not want to use an activation function for the output neurons, so they are free to output any range of values.

Tips: 
- The loss function to use during training is typically the mean squared error, but if you have a lot of outliers in the training set, you may prefer to use the *mean absolute error* instead. 
- Alternatively, you can use the Huber loss, which is a combination of both.


![cheat paper](../docs/cheat-paper.png)

## Keras

Keras is a high-level Deep Learning API that allows you to easily build, train, evaluate and execute all sorts of neural networks. Its documentation (or specification) is available at [keras.io](https://keras.io).

**Import tensorflow and keras**

In [ ]:
# ANN librarys
import tensorflow as tf
from tensorflow import keras 
from keras import metrics

In [ ]:
# split the data
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, test_size=0.20 ,shuffle=True)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

**Scale the data**

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

### MLP Keras model (Sequential API)

A Sequential model is the simplest kind of Keras model, for neural networks that are just composed of a single stack of layers, connected sequentially. This is called the Sequential API.

For this model, we will use as loss function *mean_squared_error* as suggested above and *Stochastic Gradient Descent* as optimizer.

Please note that the output layer has a single neuron (since we only want to predict a single value) and uses no activation function.

In [ ]:
# the output layer has only one neuron 
model1 = keras.models.Sequential([
                                 keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
                                 keras.layers.Dense(1) 
])

model1.summary()

In [ ]:
model1.compile(loss="mean_squared_error", optimizer="sgd")

learn_history1 = model1.fit(X_train_scaled, y_train, epochs=20,
                    validation_data=(X_valid_scaled, y_valid))

In [ ]:
plt = pd.DataFrame(learn_history1.history).plot(figsize=(8, 5))
plt.grid(True)
plt.set_ylim(0, 1) # set the vertical range to [0-1]
print(f'\nThe mse for the test set is {model1.evaluate(X_test_scaled,y_test)}')

**The training and validation loss values are decreasing as we can see above.
Moreover, the validation curve is quite close to the training curve, which means that there is not too much overfitting.**

You can also predict the test set by running *model.predict(data)*.

In [ ]:
# predict in three examples
n = 3
predict = model1.predict(X_test_scaled[:n])
true_val = y_test[:n]
for i in range(n):
    print(f'The model predicted {predict[i]} and the true value is {true_val[i]}')

### MLP Keras model (Functional API)

Here we are going to use a Functional API. Known as *Wide & Deep* neural network, this architecture makes it possible for the neural network to learn both deep patterns (using the deep path) and simple rules (through the short path). In contrast, regular MLP forces all the data to flow through the full stack of layers, thus simple patterns in the data may end up being distorted by this sequence of transformations.

![wide and deep](../docs/wide&deep.png)


In [ ]:
input = keras.layers.Input(shape=X_train.shape[1:])

hidden1 = keras.layers.Dense(30, activation="relu")(input)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)

concat = keras.layers.Concatenate()([input, hidden2])
output = keras.layers.Dense(1)(concat)

model2 = keras.models.Model(inputs=[input], outputs=[output])

Everything is exactly like earlier, so we challenge you to compile and fit the model to the data and compare it with the results above.

In [ ]:
# TODO: execute model2.compile() and model2.fit() exactly like before

In [2]:
# TODO: plot the loss curves exactly like before

#### Some interessting features in keras

Keras allows you to separate inputs, they might even overlap, which allows for the build of more complex networks. 

With Keras, you can also build models to handle different outputs. Imagine you want to predict the value of a house but also classify it as young or old. This is both a *Regression* task and a *Classification* task. Another use case for different outputs is to perform regularization techniques to reduce overfitting.

**Note: the example that follows is only for educational purposes, it is not optimized.**

**Let's imagine that we also want to classify a house as:**

    - 0 (< 25 years>) 
    - 1 (> 25 year and 50 years)
    - 2 (> 50 year)

In [ ]:
# Creating the new labels

def Classify_house_age (data):
    age_labels = []
    for datapoint in data:
        age = datapoint[1]
        label = 0
        if (age > 25 and age < 50):
            label = 1
        else:
             label = 2
        age_labels.append(label)

    return np.asarray(age_labels)

y_train_age = Classify_house_age(X_train)
y_test_age = Classify_house_age(X_test)
y_valid_age = Classify_house_age(X_valid)



**As we are going to use separate inputs to showcase this Keras feature, we need to create an input of the form:**

    input = [input_A,input_B]

In [ ]:
# Create new train valid and test set
X_train_A, X_train_age = X_train_scaled[:,:], X_train_scaled[:, 1]
X_valid_A, X_valid_age = X_valid_scaled[:, :], X_valid_scaled[:, 1]
X_test_A, X_test_age = X_test_scaled[:,:], X_test_scaled[:, 1]
X_new_A, X_new_age = X_test_A[:3], X_test_age[:3]

In [ ]:
input_A = keras.layers.Input(shape=[8])
input_B = keras.layers.Input(shape=[1])
hidden1 = keras.layers.Dense(30, activation="relu", kernel_initializer='he_normal')(input_A)
hidden2 = keras.layers.Dense(30, activation="relu", kernel_initializer='he_normal')(hidden1)
hidden3 = keras.layers.Dense(30, activation="relu", kernel_initializer='he_normal')(input_B)
concat = keras.layers.concatenate([hidden2,hidden3])

**Now we are going to build the output layer for the regressor.**

As we only want to give a single result as a output we will use 1 neuron in this output layer.

In [ ]:
regressor = keras.layers.Dense(1)(concat)

**Now we are going to build the output layer for the classifier.**

As we want to classify between 3 different categories we will use 3 neurons in this layer as well as an activation function called *softmax* indicated for this kind of tasks.

In [ ]:
classifier = keras.layers.Dense(3, activation='softmax')(hidden2)

**Now let's compile the model.**

Each output will need its loss function, so when we compile the model we should pass a list of losses. As you know and as shown above for a regression problem we use error metrics to classify the *loss* between *epochs*. 

For classification several metrics are depending on the type and range of the labels, for this example, we will use *sparse_categorical_crossentropy* as the loss function.

In [ ]:
model3 = keras.models.Model(inputs=[input_A, input_B],
                           outputs=[regressor, classifier])
model3.compile(loss=["mse","sparse_categorical_crossentropy"], optimizer="sgd")

In [ ]:
# Fiting the model 
learn_history3 = model3.fit([X_train_A, X_train_age], [y_train, y_train_age], epochs=30, verbose=2,
                        validation_data=((X_valid_A, X_valid_age), [y_valid, y_valid_age]))

In [ ]:
plt = pd.DataFrame(learn_history3.history).plot(figsize=(8, 5))
plt.grid(True)
plt.set_ylim(0, 1) # set the vertical range to [0-1]
print(f'\nThe mse for the test set is {model1.evaluate(X_test_scaled,y_test)}')

**When we evaluate the model, Keras will return the total loss, as well as all the individual losses.**

In [ ]:
total_loss, main_loss, aux_loss = model3.evaluate([X_test_A, X_test_age], [y_test, y_test_age])
y_pred, y_pred_age = model3.predict([X_new_A, X_new_age])

In [ ]:
n = 3
true_value = y_test[:3]
true_label = y_test_age[:3]
for i in range(n):
    print(f'The REGRESSOR model predicted {y_pred[i]} and the true value is {true_value[i]}\n')
    print(f'The CLASSIFIER model predicted {np.argmax(y_pred_age[i])} and the true label is {true_label[i]}\n')

### Save the final model 

Just like with pickle with Keras, you can save the final model. Keras will save both the model’s architecture (including every layer’s hyperparameters) and the value of all the model parameters for every layer (e.g., connection weights and biases), using the HDF5 format. 

In [ ]:
model3.save("temp/my_keras_model.h5")

In [ ]:
saved_model = keras.models.load_model("temp/my_keras_model.h5")